### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [13]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [14]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [15]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [16]:
ed = Website("https://ganitlabs.ai")
ed.links

['https://ganitlabs.ai/',
 'https://ganitlabs.ai/',
 'https://ganitlabs.ai/',
 'https://ganitlabs.ai/',
 'https://ganitlabs.ai/about-us/',
 'https://ganitlabs.ai/services/',
 'https://ganitlabs.ai/case-studies/',
 'https://ganitlabs.ai/contact-us/',
 'https://ganitlabs.ai/about-us/',
 'https://ganitlabs.ai/services/',
 'https://ganitlabs.ai/portfolio/prediction-of-recipe/',
 'https://ganitlabs.ai/wp-content/uploads/2020/12/case_study_food_beverage_1368x1026.jpg',
 'https://ganitlabs.ai/portfolio/prediction-of-recipe/',
 'https://ganitlabs.ai/portfolio/prediction-of-recipe/',
 'https://ganitlabs.ai/portfolio-cat/food-n-beverage/',
 'https://ganitlabs.ai/portfolio/predict-equipment-failure/',
 'https://ganitlabs.ai/wp-content/uploads/2018/06/case_study_oil_gas.jpg',
 'https://ganitlabs.ai/portfolio/predict-equipment-failure/',
 'https://ganitlabs.ai/portfolio/predict-equipment-failure/',
 'https://ganitlabs.ai/portfolio-cat/oil-gas/',
 'https://ganitlabs.ai/portfolio/machine-learning-in-

In [17]:
print(ed.get_contents())

Webpage Title:
Home - Ganit Labs
Webpage Contents:
Ganit Labs
Machine Learning, Deep Learning, AI, Data Science. Train the Machine to Think.
Home
About
Services
Case Studies
Contact
Machine Learning, Deep Learning, Artificial Intelligence
Train the machine to think
Sorry, your browser does not support inline SVG.
What we do
Our mission is to enable businesses adopt Artificial Intelligence (AI) to stay agile and competitive through incremental and transformative impacts.
Our objective is to help your business harness the power of Machine Learning (ML), Neural Nets, Deep Learning and statistical inference to enhance efficiencies and improve the bottom line.
We develop and implement
custom
AI solutions based on your specific business needs. And work with you from inception, development to final deployment.
LEARN MORE
AI, a data driven approach
Do machines have AI?
EXPERTISE
Customized, end-to-end, reliable, and scalable solution, from design to deployment
Data Science - AI & Machine Learn

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [18]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [19]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [20]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [21]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://ganitlabs.ai - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://ganitlabs.ai/
https://ganitlabs.ai/
https://ganitlabs.ai/
https://ganitlabs.ai/
https://ganitlabs.ai/about-us/
https://ganitlabs.ai/services/
https://ganitlabs.ai/case-studies/
https://ganitlabs.ai/contact-us/
https://ganitlabs.ai/about-us/
https://ganitlabs.ai/services/
https://ganitlabs.ai/portfolio/prediction-of-recipe/
https://ganitlabs.ai/wp-content/uploads/2020/12/case_study_food_beverage_1368x1026.jpg
https://ganitlabs.ai/portfolio/prediction-of-recipe/
https://ganitlabs.ai/portfolio/prediction-of-recipe/
https://ganitlabs.ai/portfolio-cat/food-n-beverage/
https://ganitlabs.ai/portfolio/predict-equipment-failure/
https://ganitlabs.ai/wp-content/uploads/2018/06/case_study_oil_gas.jpg

In [22]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [23]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [24]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'},
  {'type': 'api page', 'url': 'https://anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'},
  {'type': 'news page', 'url': 'https://anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [25]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [26]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for Enterprise
Sep 4, 2024
Alignment
·
Research
Constitutional AI: Harmlessness from

In [27]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [28]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [29]:
print(get_brochure_user_prompt("Anthropic", "https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}
You are looking at a company called: Anthropic
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}


# Anthropic Company Brochure

---

### **About Us**
Anthropic is a pioneering AI safety and research company based in San Francisco, dedicated to creating reliable, interpretable, and steerable AI systems. We believe in the profound impact AI will have on the world and are committed to building systems that people can trust, while also generating research that addresses both opportunities and risks associated with AI technology.

### **Our Mission**
At Anthropic, our goal is to ensure that transformative AI contributes positively to society. We engage in frontier research and employ a variety of safety techniques to build AI systems that are not only innovative but also aligned with human values.

---

### **Our Products**
#### **Claude**
Meet **Claude 3.5 Sonnet**, our most advanced AI model yet. With its capabilities, companies can streamline processes and create new revenue streams. Claude also serves as a collaborative tool for teams, empowering them to work smarter and more creatively.

- **API Access**: Start building with Claude to enhance your operational efficiency.
- **Enterprise Solutions**: Claude for Enterprise offers tailored solutions for businesses, enabling you to integrate advanced AI into your operations seamlessly.

---

### **Company Culture**
At Anthropic, we foster a high-trust environment where collaboration and transparency are paramount. Our values include:

- **Mission-Centric**: Our focus is on creating impactful AI that contributes to societal flourishing.
- **Collaboration**: We work as one large team, blending diverse perspectives from fields like machine learning, physics, and public policy.
- **Pragmatism**: We prioritize robust solutions and believe in testing simple ideas before pursuing more complex ones.
- **Empirical Approach**: We value direct evidence of ability and encourage exploration and innovation through research.

Our team is filled with experts from various domains, united by the common goal of making AI safe and beneficial.

---

### **Customer Base**
Anthropic serves businesses, nonprofits, and civil society groups by providing AI models and tools that facilitate innovation and efficiency. Some of our notable clients, including North Highland and IG Group, have reported significant improvements in their workflows and productivity, thanks to Claude’s capabilities.

---

### **Careers at Anthropic**
We are constantly looking for talented individuals to join our team. We believe our employees' well-being is critical to our collective success and offer comprehensive benefits, including:

- Health, dental, and vision insurance
- 22 weeks of paid parental leave
- Flexible paid time off and generous mental health support
- Competitive salary and equity packages
- Daily meals/snacks at the office and wellness stipends

Our hiring process is designed to be inclusive and thorough, aiming to bring in diverse talents who can contribute to our mission, regardless of their background or prior experience in machine learning.

---

### **Join Us**
If you’re passionate about building safer AI systems and want to be part of a mission-driven company, we invite you to join our team. Explore open positions on our [Careers page](#).

---

### **Stay Connected**
Follow us on social media:
- [Twitter](#)
- [LinkedIn](#)
- [YouTube](#)

For inquiries or to learn more about our work, please visit our [website](#).

---

**Anthropic**: Building AI that you can trust. Together, let's ensure that transformative AI benefits humanity.

In [32]:
create_brochure("Ganit Labs", "https://ganitlabs.ai")

Found links: {'links': [{'type': 'about page', 'url': 'https://ganitlabs.ai/about-us/'}, {'type': 'services page', 'url': 'https://ganitlabs.ai/services/'}, {'type': 'case studies page', 'url': 'https://ganitlabs.ai/case-studies/'}, {'type': 'contact page', 'url': 'https://ganitlabs.ai/contact-us/'}]}


# Ganit Labs Brochure

---

## **Welcome to Ganit Labs**

**Machine Learning, Deep Learning, AI, Data Science. Train the Machine to Think.**

---

### **About Us**

At Ganit Labs, we empower businesses to become agile and competitive by harnessing the transformative power of artificial intelligence (AI). As a full-service data science and engineering company, we specialize in developing custom AI solutions tailored to meet the unique needs of our clients. Our mission is to transform challenges into opportunities, utilizing cutting-edge technologies such as machine learning (ML), deep learning (DL), and neural networks.

---

### **Our Services**

#### **Customized, End-to-End Solutions:**
- **Artificial Intelligence (AI) & Machine Learning (ML):** Advanced techniques for predictive, descriptive, and prescriptive analytics focused on explainable AI.
- **Data Engineering:** Collaboration with data scientists to build reliable ETL pipelines and architect scalable distributed systems.
- **Cloud & Edge (IoT):** Implementation of solutions in the cloud and on the edge based on specific business needs and data sensitivity.
- **Predictive Analytics:** Accurate event predictions using advanced statistics and machine learning techniques.
- **Computer Vision:** Actionable insights from images using state-of-the-art deep learning techniques.
- **Natural Language Processing (NLP):** Understanding structured data such as text and speech through machine learning.

---

### **Our Expertise**

- **Experience & Knowledge:** Our leadership has a combined experience of over 20 years in machine learning and technology services, with team members from prestigious institutions such as Carnegie Mellon and Stanford.
- **Domain Diversity:** Our team includes industry veterans with backgrounds in academia, finance, mechanical engineering, and health sciences, collectively enhancing our service delivery.

---

### **Company Culture**

**Innovation and Collaboration:** At Ganit Labs, we pride ourselves on a culture of collaboration and transparency, engaging closely with our partners through every step of the AI implementation process. We believe in the power of diverse ideas and pragmatic research, ensuring that our solutions are not only effective but also comprehensible to our clients.

**Quality & Improvement:** We are committed to continuous improvement, maintaining a long-term focus on delivering quality work that meets the evolving needs of our stakeholders.

---

### **Our Clients**

Ganit Labs has worked with a diverse range of clients, including publicly traded companies in sectors such as:
- Oil & Gas
- Food & Beverage
- Health Care & Genomics
- Manufacturing and research laboratories

**Client Testimonial:**
“Ganit Labs provided a simple, accurate, and elegant AI-based solution, customized for our needs.” - Oilfield service provider

---

### **Career Opportunities**

Join our team of talented professionals dedicated to solving complex problems with innovative AI solutions. We are on the lookout for passionate individuals eager to contribute to exciting projects in a dynamic and supportive environment.

---

### **Get in Touch**

**Office Locations:**
- 10042 Monroe Dr, Dallas, TX 75229
- 152 W 57th St, New York, NY 10019

**Contact Us:**
- Phone: +1 972 559 9669
- Email: [info@ganitlabs.ai](mailto:info@ganitlabs.ai)

**Follow Us:**
[LinkedIn](#) | [Twitter](#) | [Facebook](#)

---

**Ganit Labs – Where Human Intelligence Meets Machine Learning**  
*© 2024 Ganit Labs. All rights reserved.*  
*Transforming data challenges into strategic advantages, one AI solution at a time.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [33]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [34]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic: Pioneering the Future of Safe AI

## Overview
Anthropic is a cutting-edge AI safety and research company based in San Francisco, dedicated to developing reliable, interpretable, and steerable AI systems. Our goal is to build AI technologies that can be safely integrated into society, ensuring that their transformative potential is harnessed responsibly.

## Our Vision
At Anthropic, we recognize that artificial intelligence can have profound effects on the world. We are committed to generating research that illuminates both the opportunities and risks associated with AI, driving advancements that prioritize safety and ethical considerations in AI systems.

## Meet Claude
Our flagship product, **Claude**, is an advanced AI model designed to assist companies in increasing efficiency and innovating new revenue streams. With various iterations such as Claude 3.5 Sonnet, we continue to push the boundaries of AI capabilities while maintaining a stringent focus on safety.

## Interdisciplinary Team
Anthropic's strength lies in our diverse and collaborative team, which includes experts in machine learning, physics, policy, and product development. This interdisciplinary approach fosters a holistic understanding of AI technologies, enabling us to address safety concerns from multiple angles.

## Company Culture
Anthropic fosters a unique work environment characterized by:
- **High Trust**: We assume good faith, prioritize honesty, and encourage intellectual openness.
- **Collaboration**: We believe in working as "one big team," integrating insights from all areas of our organization to achieve our mission.
- **Simplicity**: We focus on pragmatic solutions, preferring simple and effective methods of operation before exploring complex alternatives.

## Careers at Anthropic
We are always looking for passionate individuals who are eager to help us build the future of safe AI. Our hiring process is designed to identify unique strengths and diverse expertise, making our team a vibrant space for innovation. 

### What We Offer
- **Health & Wellness**: Comprehensive health plans, mental health support, and parental leave.
- **Compensation & Benefits**: Competitive salaries, flexible wellness stipends, and robust retirement plans.
- **Supportive Work Environment**: Opportunities for remote work and a culture that values personal growth and well-being.

## Customers
Our AI solutions, particularly Claude, are utilized by businesses to streamline operations in various sectors, including engineering, marketing, and customer support. Major companies are leveraging Claude's capabilities to enhance productivity and drive innovation.

## Join Us
Ready to shape the future of AI with us? Check our [careers page](https://anthropic.com/careers) to explore available opportunities and apply to join our mission of making AI systems that people can trust.

---
**Contact us**: Reach us at [press@anthropic.com](mailto:press@anthropic.com) for press inquiries. 

Follow our journey and become part of a team that’s committed to creating safe AI for a better tomorrow!

In [35]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


# Hugging Face Brochure

**Welcome to Hugging Face – The AI Community Building the Future**

---

## About Us

Hugging Face is a leading platform that fosters collaboration among machine learning enthusiasts and professionals. Our community-centric approach is designed to accelerate the development of state-of-the-art AI models and datasets while supporting developers across various modalities, including text, image, video, audio, and 3D.

---

## Services We Offer

- **Models:** With over 400,000 models available, users can explore cutting-edge advancements in AI.
- **Datasets:** Access to more than 100,000 datasets for all machine learning needs.
- **Spaces:** Collaborate on applications with nearly 150,000 spaces for experimentation and innovation.
- **Enterprise Solutions:** Tailored services for organizations, featuring enterprise-grade security, dedicated support, and pricing starting at **$20/user/month**.

---

## Our Customers

Join a progressive community of innovators! More than **50,000 organizations** trust Hugging Face, including:
- Amazon Web Services
- Microsoft
- Google
- Grammarly
- Toyota Research Institute

---

## Company Culture

At Hugging Face, we believe in open collaboration and transparency. Our culture encourages creativity, innovation, and community engagement. We are committed to building a diverse and inclusive environment where every member can thrive and contribute their unique skills.

---

## Careers

We are constantly on the lookout for talented individuals to join our passionate team. Whether you are an AI researcher, software developer, or marketing professional, Hugging Face offers exciting opportunities for career growth in a supportive environment. 

**Current Openings:** Visit our [careers page](#) to explore available positions and join us in revolutionizing the AI landscape.

---

## Join Us

Whether you are a developer, an enterprise, or a job seeker, Hugging Face is where the future of AI is being built. 

- **Explore our models and datasets**: [Visit Hugging Face Hub](#)
- **Connect with our community**: Follow us on [Twitter](#), [LinkedIn](#), or [Discord](#).

Together, let’s shape the future of AI!

--- 

**Contact Us:**  
For inquiries, please reach out at: [info@huggingface.co](mailto:info@huggingface.co)

© Hugging Face | All rights reserved.